In [10]:
import common.ipynb_importer
from db.pg.pg_00_common import *

cursor = pg_connect()

# Exception

语法：
```
<<label>>
declare
   ...
begin
    ...
exception
    when condition [or condition...] then
       handle_exception;
   [when condition [or condition...] then
       handle_exception;]
   [when others then
       handle_other_exceptions;
   ]
end;
```

## no_data_found 异常处理

In [2]:
sql = """
do
$$
declare
	rec record;
	v_film_id int = 2000;
begin
	-- select a film 
	select film_id, title 
	into strict rec
	from film
	where film_id = v_film_id;
end;
$$
language plpgsql;
"""
cursor.execute(sql)

NoDataFound: 查询没有返回记录
CONTEXT:  在SQL语句的第7行的PL/pgSQL函数inline_code_block

In [4]:
sql = """
do
$$
declare
	rec record;
	v_film_id int = 2000;
begin
	-- select a film 
	select film_id, title 
	into strict rec
	from film
	where film_id = v_film_id;
        -- catch exception
	exception 
	   when no_data_found then 
	      raise exception 'film % not found', v_film_id;
end;
$$;
"""
cursor.execute(sql)

RaiseException: film 2000 not found
CONTEXT:  在RAISE的第14行的PL/pgSQL函数inline_code_block

## too_many_rows 异常处理

In [7]:
# select into 语句返回了超多一行的数据
sql = """
do
$$
declare
	rec record;
begin
	-- select film 
	select film_id, title 
	into strict rec
	from film
	where title LIKE 'A%';
	
	exception 
	   when too_many_rows then
	      raise exception 'Search query returns too many rows';
end;
$$;
"""
cursor.execute(sql)

RaiseException: Search query returns too many rows
CONTEXT:  在RAISE的第13行的PL/pgSQL函数inline_code_block

## Multiple Exceptions

In [9]:
sql = """
do
$$
declare
	rec record;
	v_length int = 90;
begin
	-- select a film 
	select film_id, title 
	into strict rec
	from film
	where length = v_length;
	
    -- catch exception
	exception 
	   when sqlstate 'P0002' then 
	      raise exception 'film with length % not found', v_length;
	   when sqlstate 'P0003' then 
	      raise exception 'The with length % is not unique', v_length;
end;
$$;
"""
cursor.execute(sql)

RaiseException: The with length 90 is not unique
CONTEXT:  在RAISE的第17行的PL/pgSQL函数inline_code_block

## 将异常作为 SQLSTATE 来捕获

In [11]:
sql = """
do
$$
declare
	rec record;
	v_length int = 30;
begin
	-- select a film 
	select film_id, title 
	into strict rec
	from film
	where length = v_length;
	
        -- catch exception
	exception 
	   when sqlstate 'P0002' then 
	      raise exception 'film with length % not found', v_length;
	   when sqlstate 'P0003' then 
	      raise exception 'The with length % is not unique', v_length;
end;
$$;
"""
cursor.execute(sql)

RaiseException: film with length 30 not found
CONTEXT:  在RAISE的第15行的PL/pgSQL函数inline_code_block